In [1]:
from swarm import Swarm, Agent

client = Swarm()

def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Chinese.",
)

response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

print(response.messages[-1]["content"])

正在将您转接到B代理。


In [16]:
response.messages

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_NO9cMSdceJXkW7KTMeK3j8Jm',
    'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'},
    'type': 'function'}],
  'refusal': None,
  'sender': 'Agent A'},
 {'role': 'tool',
  'tool_call_id': 'call_NO9cMSdceJXkW7KTMeK3j8Jm',
  'tool_name': 'transfer_to_agent_b',
  'content': '{"assistant": "Agent B"}'},
 {'content': '对不起，我无法将您直接转接给特定的代理人，但我可以帮助您解决问题或回答您的问题。请告诉我您需要什么帮助。',
  'role': 'assistant',
  'function_call': None,
  'tool_calls': None,
  'refusal': None,
  'sender': 'Agent B'}]

In [2]:
client.memory.model_dump()

{'memory': [{'messages': [{'content': None,
     'role': 'assistant',
     'function_call': None,
     'tool_calls': [{'id': 'call_vJvmKrQZU2v7FVLgbX3TVz9b',
       'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'},
       'type': 'function'}],
     'refusal': None,
     'sender': 'Agent A'},
    {'role': 'tool',
     'tool_call_id': 'call_vJvmKrQZU2v7FVLgbX3TVz9b',
     'tool_name': 'transfer_to_agent_b',
     'content': '{"assistant": "Agent B"}'},
    {'content': '正在将您转接到B代理。',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': None,
     'refusal': None,
     'sender': 'Agent B'}],
   'agent': {'name': 'Agent B',
    'model': 'gpt-4o',
    'instructions': 'Only speak in Chinese.',
    'functions': [],
    'tool_choice': None,
    'parallel_tool_calls': True},
   'context_variables': {},
   'timestamp': '2024-10-13 23:07:21'}]}

In [10]:
from swarm.types import Result

sales_agent = Agent(name="Sales Agent")

def talk_to_sales():
   print("Hello, World!")
   return Result(
       value="Done",
       agent=sales_agent,
       context_variables={"department": "sales"}
   )

agent = Agent(functions=[talk_to_sales])

response = client.run(
   agent=agent,
   messages=[{"role": "user", "content": "Transfer me to sales"}],
   context_variables={"user_name": "John"}
)
print(response.agent.name)
print(response.context_variables)

Hello, World!
Sales Agent
{'user_name': 'John', 'department': 'sales'}


In [11]:
import json
from swarm import Swarm

def process_and_print_streaming_response(response):
    content = ""
    last_sender = ""

    for chunk in response:
        if "sender" in chunk:
            last_sender = chunk["sender"]

        if "content" in chunk and chunk["content"] is not None:
            if not content and last_sender:
                print(f"\033[94m{last_sender}:\033[0m", end=" ", flush=True)
                last_sender = ""
            print(chunk["content"], end="", flush=True)
            content += chunk["content"]

        if "tool_calls" in chunk and chunk["tool_calls"] is not None:
            for tool_call in chunk["tool_calls"]:
                f = tool_call["function"]
                name = f["name"]
                if not name:
                    continue
                print(f"\033[94m{last_sender}: \033[95m{name}\033[0m()")

        if "delim" in chunk and chunk["delim"] == "end" and content:
            print()  # End of response message
            content = ""

        if "response" in chunk:
            return chunk["response"]


def pretty_print_messages(messages) -> None:
    for message in messages:
        if message["role"] != "assistant":
            continue

        # print agent name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # print response, if any
        if message["content"]:
            print(message["content"])

        # print tool calls in purple, if any
        tool_calls = message.get("tool_calls") or []
        if len(tool_calls) > 1:
            print()
        for tool_call in tool_calls:
            f = tool_call["function"]
            name, args = f["name"], f["arguments"]
            arg_str = json.dumps(json.loads(args)).replace(":", "=")
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


In [17]:
from examples.triage_agent.agents import triage_agent

In [18]:
client = Swarm()

messages = []
agent = triage_agent
stream = True
debug = False

while True:
    user_input = input("\033[90mUser\033[0m: ")
    messages.append({"role": "user", "content": user_input})

    response = client.run(
        agent=agent,
        messages=messages,
        context_variables={},
        stream=stream,
        debug=debug,
    )

    if stream:
        response = process_and_print_streaming_response(response)
    else:
        pretty_print_messages(response.messages)

    messages.extend(response.messages)
    agent = response.agent

Triage Agent: Hello! How can I assist you today?
Triage Agent: transfer_to_sales()
Sales Agent: I'm here to help with all your bee-related needs! Whether you're looking to buy bees, learn about hive equipment, or anything else bee-related, I'm excited to assist. What can I help you with today?
Sales Agent: That's fantastic! Investing in a bee hive is a wonderful step, whether you're starting a new hobby or expanding your apiary. We have a variety of hives to suit every need, from classic Langstroth hives to top-bar hives and even the beautiful Warré hives. 

Each hive type has its own unique benefits:
- **Langstroth Hives:** Known for ease of access and management, especially popular among beekeepers.
- **Top-Bar Hives:** Great for beginners; they offer a more natural beekeeping experience.
- **Warré Hives:** Designed to mimic a wild bee colony, promoting natural bee behaviors.

What type of hive are you interested in, or would you like more information on each type? Let me know, and t

KeyboardInterrupt: Interrupted by user

In [21]:
print(len(client.memory.memory))
client.memory.model_dump()

25


{'memory': [{'messages': [{'content': 'Hello! How can I assist you today?',
     'sender': 'Triage Agent',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': None}],
   'agent': {'name': 'Triage Agent',
    'model': 'gpt-4o',
    'instructions': "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
    'functions': [<function examples.triage_agent.agents.transfer_to_sales()>,
     <function examples.triage_agent.agents.transfer_to_refunds()>],
    'tool_choice': None,
    'parallel_tool_calls': True},
   'context_variables': {},
   'timestamp': '2024-10-13 22:44:39'},
  {'messages': [{'content': '',
     'sender': 'Triage Agent',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': [{'function': {'arguments': '{}',
        'name': 'transfer_to_sales'},
       'id': 'call_kw0ZIuZUwjNHfhL5BHLiRv0S',
       'type': 'function'}]},
    {'role': 'tool',
     'tool_call_id': 'call_kw0ZIuZUwj